In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from skimage.util import montage
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Conv2D, LeakyReLU, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
import os
import pdb
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

2022-12-09 11:50:20.465976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path = '../PKLot/PKLotSegmented_resized/'

In [3]:
# load train data
print('indexing train data')
train_img = []
train_label = []
for folder in os.listdir(path):
    if folder[0] != '.':
        for weather in os.listdir(os.path.join(path, folder)):
            if weather[0] != '.':
                for date in os.listdir(os.path.join(path, folder, weather))[:-4]:
                    if date[0] != '.':
                        for occupied in os.listdir(os.path.join(path, folder, weather, date)):
                            if occupied[0] != '.':
                                for img in os.listdir(os.path.join(path, folder, weather, date, occupied)):
                                    train_label.append(occupied)
                                    train_img.append(os.path.join(path, folder, weather, date, occupied, img))

# load test data
print('indexing test data')
test_img = []
test_label = []
for folder in os.listdir(path):
    if folder[0] != '.':
        for weather in os.listdir(os.path.join(path, folder)):
            if weather[0] != '.':
                for date in os.listdir(os.path.join(path, folder, weather))[-4:-1]:
                    if date[0] != '.':
                        for occupied in os.listdir(os.path.join(path, folder, weather, date)):
                            if occupied[0] != '.':
                                for img in os.listdir(os.path.join(path, folder, weather, date, occupied)):
                                    test_label.append(occupied)
                                    test_img.append(os.path.join(path, folder, weather, date, occupied, img))

# load validation data
print('indexing validation data')
valid_img = []
valid_label = []
for folder in os.listdir(path):
    if folder[0] != '.':
        for weather in os.listdir(os.path.join(path, folder)):
            if weather[0] != '.':
                for date in os.listdir(os.path.join(path, folder, weather))[-1:]:
                    if date[0] != '.':
                        for occupied in os.listdir(os.path.join(path, folder, weather, date)):
                            if occupied[0] != '.':
                                for img in os.listdir(os.path.join(path, folder, weather, date, occupied)):
                                    valid_label.append(occupied)
                                    valid_img.append(os.path.join(path, folder, weather, date, occupied, img))

indexing train data
indexing test data
indexing validation data


In [4]:
numoutputs = len(np.unique(train_label))
_, t = np.unique(train_label, return_inverse=True)
t_train = to_categorical(t, numoutputs)

_, t = np.unique(test_label, return_inverse=True)
t_test = to_categorical(t, numoutputs)

_, t = np.unique(valid_label, return_inverse=True)
t_valid = to_categorical(t, numoutputs)

In [5]:
# create dataframe to load with tf datagenerator
train_df = pd.DataFrame(list(zip(train_img, t_train)), columns =['img_path', 'label'])
test_df = pd.DataFrame(list(zip(test_img, t_test)), columns =['img_path', 'label'])
valid_df = pd.DataFrame(list(zip(valid_img, t_valid)), columns =['img_path', 'label'])

In [6]:
img_size = (90, 90)

datagen = ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="./",
    x_col="img_path",
    y_col="label",
    #subset="training",
    batch_size=1000,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=img_size,
    validate_filenames=False
    )
    
valid_generator=datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory="./",
    x_col="img_path",
    y_col="label",
    #subset="training",
    batch_size=1000,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=img_size,
    validate_filenames=False
    )

# test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="./",
    x_col="img_path",
    y_col="label",
    #subset="training",
    batch_size=1000,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=img_size,
    validate_filenames=False
    )

# fix for bug in keras with 'raw' input in datagenerators:
# https://stackoverflow.com/questions/64978209/valueerror-when-trying-to-execute-model-fit-failed-to-convert-a-numpy-array/
train_generator._targets = np.stack(train_generator._targets)
valid_generator._targets = np.stack(valid_generator._targets)
test_generator._targets = np.stack(test_generator._targets)

Found 489852 non-validated image filenames.
Found 57115 non-validated image filenames.
Found 148884 non-validated image filenames.


In [7]:
# create neural network
model = Sequential()
model.add(Input((*img_size, 3)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", name='Conv2D_1'))
model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", name='Conv2D_2'))
model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(units=100, name='dense_1'))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(units=10, name='dense_2'))
model.add(LeakyReLU(alpha=0.3))
# model.add(Dense(units=100, activation='sigmoid', name='hidden2'))
# model.add(Dense(units=50, activation='relu', name='hidden3'))
# model.add(Dense(units=50, activation='relu', name='hidden4'))
model.add(Dense(units=2, activation='softmax', name='output'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1 (Conv2D)           (None, 90, 90, 32)        896       


2022-12-09 11:50:35.516270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


                                                                 
 leaky_re_lu (LeakyReLU)     (None, 90, 90, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 32)       0         
 )                                                               
                                                                 
 Conv2D_2 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3200)              0         
                                                                 
 dense_1 (

In [8]:
model.compile(
    optimizer= Adam(learning_rate=0.00001),
    loss='mse',
    metrics=['accuracy']
    )

In [9]:
# checkpoint = ModelCheckpoint("best_simple_conv_weights_dense_leaky_relu", monitor='accuracy', verbose=1,
#     save_best_only=False, mode='auto')

model.fit(train_generator, validation_data=valid_generator) #, callbacks=[checkpoint])

100/490 [=====>........................] - ETA: 35:51 - loss: 0.2250 - accuracy: 0.7013

KeyboardInterrupt: 

In [ ]:
# save weights
model.save('simple_conv_weights_dense_leaky_relu')

In [ ]:
model.evaluate(test_generator)